In [ ]:
import numpy as np

# Constants
L_v = 2.5e6        # Latent heat of vaporization (J/kg)
R_v = 461.5        # Specific gas constant for water vapor (J/(kg·K))
R_air = 287        # Specific gas constant for dry air (J/(kg·K))
T_0 = 273.15       # Reference temperature (K)
e_0 = 611.2        # Reference vapor pressure at T_0 (Pa)
P_atm = 101325     # Atmospheric pressure (Pa)
D = 2.5e-5         # Diffusion coefficient for water vapor in air (m²/s)
lambda_layer = 0.01  # Boundary layer thickness (meters)

# Chamber dimensions
L = 3              # Side length (meters)
H = 9              # Height (meters)

# Wall temperatures (°C)
T_b = 30           # Bottom boundary temperature
T_t = 10           # Top boundary temperature
T_s = 30           # Temperature for three sidewalls
T_s_prime = 10     # Temperature for one unique sidewall

# Time period in hours
time_period_hours = 1 


# Calculate saturation vapor pressure
def saturation_vapor_pressure(T_celsius):
    T_kelvin = T_celsius + 273.15
    return e_0 * np.exp((L_v / R_v) * (1 / T_0 - 1 / T_kelvin))

# Saturation vapor pressures
e_b = saturation_vapor_pressure(T_b)
e_t = saturation_vapor_pressure(T_t)
e_s = saturation_vapor_pressure(T_s)
e_s_prime = saturation_vapor_pressure(T_s_prime)

# Mean temperature
T_mean = (T_b + T_t + 3 * (H / L) * T_s + (H / L) * T_s_prime) / (2 + 4 * (H / L))
T_mean_K = T_mean + 273.15

# Mean vapor pressure
e_mean = (e_b + e_t + 3 * (H / L) * e_s + (H / L) * e_s_prime) / (2 + 4 * (H / L))

# Air density
rho_a = P_atm / (R_air * T_mean_K)

# Supersaturation
e_s_mean = saturation_vapor_pressure(T_mean)
supersaturation = (e_mean / e_s_mean) - 1

# Wall areas
A_b = L * L
A_t = L * L
A_s = H * L

# Compute q_v 
numerator = (
     e_b / (rho_a * R_v * (T_b + 273.15)) * A_b
    +  e_t / (rho_a * R_v * (T_t + 273.15)) * A_t
    + 3 *  e_s / (rho_a * R_v * (T_s + 273.15)) * A_s
    +  e_s_prime / (rho_a * R_v * (T_s_prime + 273.15)) * A_s
)

denominator = (
     A_b + A_t + 4 * A_s
)

q_v = numerator / denominator

# Function to calculate flux and mass flow rate
def calculate_flux_and_mass_flow(q_v, e_sat, T_wall, A):
    T_wall_K = T_wall + 273.15
    qv_sat = e_sat / (rho_a * R_v * T_wall_K)
    J = (rho_a * D / lambda_layer) * (qv_sat - q_v)  # Flux (kg/m²*s)
    mdot = J * A  
    return J, mdot, qv_sat

# Calculate fluxes and mass flow rates for each wall
J_b, mdot_b, qv_sat_b = calculate_flux_and_mass_flow(q_v, e_b, T_b, A_b)
J_t, mdot_t, qv_sat_t = calculate_flux_and_mass_flow(q_v, e_t, T_t, A_t)
J_s, mdot_s, qv_sat_s = calculate_flux_and_mass_flow(q_v, e_s, T_s, A_s)
mdot_s_total = 3 * mdot_s
J_s_prime, mdot_s_prime, qv_sat_s_prime = calculate_flux_and_mass_flow(q_v, e_s_prime, T_s_prime, A_s)

# Net mass flow rate
Mdot_net = mdot_b + mdot_t + mdot_s_total + mdot_s_prime


# Calculate evaporated mass flow rate
net_evaporation_rate = mdot_b + mdot_s_total  # Evaporation contributions from bottom and side walls

def calculate_water_volume(evaporation_rate, time_hours):
    time_seconds = time_hours * 3600  
    water_density = 1000  # Density of water in kg/m³
    total_mass = evaporation_rate * time_seconds  # Total mass of evaporated water in kg
    volume_liters = total_mass / water_density * 1000  # Convert m³ to liters

    return volume_liters, time_seconds
    
# Calculate water volume
water_volume_liters, time_period_seconds = calculate_water_volume(net_evaporation_rate, time_period_hours)

# Conversion from liters to gallons
liter_to_gallon = 0.264172  # Conversion factor
water_volume_gallons = water_volume_liters * liter_to_gallon


# Output results
print(f"Mean vapor pressure: {e_mean:.2f} Pa")
print(f"Mean temperature: {T_mean:.2f} °C")
print(f"Mixing ratio (q_v): {q_v:.8f} kg/kg")
print(f"Saturation vapor pressure at mean temperature: {e_s_mean:.2f} Pa")
print(f"Supersaturation: {supersaturation:.4f}")
print(f"Bottom Wall: J = {J_b:.8f} kg/m²/s, mdot = {mdot_b:.8f} kg/s")
print(f"Top Wall: J = {J_t:.8f} kg/m²/s, mdot = {mdot_t:.8f} kg/s")
print(f"Three Side Walls: J = {J_s:.8f} kg/m²/s, mdot_total = {mdot_s_total:.8f} kg/s")
print(f"Unique Side Wall: J = {J_s_prime:.8f} kg/m²/s, mdot = {mdot_s_prime:.8f} kg/s")
print(f"\nNet mass flow rate: {Mdot_net:.8f} kg/s")

print(f"Water volume sourced over {time_period_hours} hours: {water_volume_liters:.2f} liters ({water_volume_gallons:.2f} gallons)")
